In [1]:
import csv
import pandas as pd
import pickle

In [2]:
data = pd.read_csv("./csv/youtrend_merged1.csv")
# concatenated data among all countries

In [3]:
# data['tags']

In [4]:
##### tags: string to set

tags_col = data['tags'].str.split('|')
tags = []
for tag_list in tags_col:
    tags.append(set(tag_list))
data_new = data.drop(['tags'], axis=1)
data_new.insert(7, 'tags', tags)

# parse dates
from dateutil.parser import isoparse
# publishing date
for idx in range(data_new.shape[0]):
    pub_date = data_new['publishedAt'][idx]
    data_new.at[idx, 'publishedAt'] = isoparse(pub_date)
# trending date
for idx in range(data_new.shape[0]):
    trend_date = data_new['trending_date'][idx]
    data_new.at[idx, 'trending_date'] = isoparse(trend_date)

In [5]:
data_new.columns

Index(['lang', 'country', 'video_id', 'title', 'publishedAt', 'channelTitle',
       'categoryId', 'tags', 'trending_date', 'view_count', 'likes',
       'dislikes', 'comment_count', 'thumbnail_link'],
      dtype='object')

In [6]:
##### Sort by trending date

data_sorted = data_new.sort_values(by=['video_id'])
video_list = list(data_sorted['video_id'].unique())
data_re = data_sorted.reset_index()

# merge trending dates of one video
# video_trend (dict): {video_id: [a list of trending dates]}
video_trend = {}
index = 0
for videoid in video_list:
    #print('id:', videoid)
    video_trend[videoid] = []
    for i in range(index, data_re.shape[0]):
        #print(data_re['video_id'][i])
        if data_re['video_id'][i]==videoid:
            video_trend[videoid].append(data_re['trending_date'][i])
        else:
            break
        index = i+1


# video_trend_sorted (dict): {video_id: [a sorted list by trending date]}
video_trend_sorted = {}
for idx in video_list:
    # dates = [isoparse(trend_date) for trend_date in video_trend[idx]]
    dates = [trend_date for trend_date in video_trend[idx]]
    video_trend_sorted[idx] = sorted(dates)

#    
index = 0
merged_trend_dates = []
for videoid in video_list:
    #print('id:', videoid)
    for i in range(index, data_re.shape[0]):
        #print(data_re['video_id'][i])
        if data_re['video_id'][i]==videoid:
            # data_re.at[i, 'trending_date'] = tuple(video_trend_sorted[videoid])
            merged_trend_dates.append(tuple(video_trend_sorted[videoid]))
        else:
            break
        index = i+1

In [7]:
data_re.insert(9, 'tuple_trending_dates', merged_trend_dates)
data_re2 = data_re.sort_values(by=['trending_date']).reset_index() # sorted by trending date, to get the latest

In [35]:
# latest_date = data_re2['trending_date'][data_re2.shape[0]-1] # datetime.datetime(2021, 3, 7, 0, 0, tzinfo=tzutc())
# # latest_date

datetime.datetime(2021, 3, 7, 0, 0, tzinfo=tzutc())

In [8]:
data_re2 = data_re2.drop(['level_0'],axis=1)
data_re2 = data_re2.drop(['index'], axis=1)
# how to remove 'level_0' and 'index' from the dict?

In [9]:
columns = data_re2.columns
attr = columns.tolist()
del attr[2]
del attr[-6]
alist = list(range(len(attr)))
attr = dict(zip(alist, attr))
with open('./index/attributes.pickle','wb') as file:
    pickle.dump(attr, file, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
# lowercase for country names
# countries = [cn.lower() for cn in data_re2.country.unique()]

In [11]:
country_idx = {}
for _, row in data_re2.iterrows():
    #print(c)
    c = row['country']
    if not c in country_idx:
        country_idx[c] = []
    country_idx[c].append(row['video_id'])
    #print(row['video_id'])

In [12]:
# country_idx

In [13]:
# yt_dict = data_re.set_index('video_id').T.to_dict('list')
yt_dict = data_re2.set_index('video_id').T.to_dict('list')
# view_count, likes, ... correspond to the latest trending date

# import pickle

# file_to_write = open("output.pickle", "wb")
# pickle.dump(yt_dict, file_to_write)
# file_to_write.close()

# yt_dict = {key=video_id: value=(a tuple of attributes)}
for key in yt_dict:
#     del yt_dict[key][:2] # delete level_0 and index from the pandas dataframe
    yt_dict[key] = tuple(yt_dict[key])

<ipython-input-13-ae3164a4b1e3>:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  yt_dict = data_re2.set_index('video_id').T.to_dict('list')


In [14]:
#### Indices
# country_idx:
#    keys: 'gb', 'mx', 'fr', 'ru', 'kr', 'us', 'in', 'jp', 'br', 'ca', 'de'
# category_idx:
#    keys: 23, 10, 24, 2, 26, 22, 17, 20, 27, 25, 28, 19, 15, 1, 29
# lang_idx:
#    keys: 'en',...,'zh-cn', 'unknown'
# trending_idx:
#    keys: 1w, 1m, 3m

# Category indices
category_idx = {}
for vid in yt_dict:
    ctid = yt_dict[vid][5] # category ID
    if not ctid in category_idx:
        category_idx[ctid] = []
    category_idx[ctid].append(vid)

# Language indices
lang_idx = {}
for vid in yt_dict:
    lg = yt_dict[vid][0] # language
    if not lg in lang_idx:
        lang_idx[lg] = []
    lang_idx[lg].append(vid)

In [36]:
# yt_dict['UNV6ZfY1Wp0']

In [16]:
# Trending date indices
from datetime import datetime, timedelta

# the latest date in the current dataset
latest_date = data_re2['trending_date'][data_re2.shape[0]-1] # datetime.datetime(2021, 2, 21, 0, 0, tzinfo=tzutc())

trending_1w = []
trending_1m = []
trending_3m = []
for vid in yt_dict:
    if latest_date < yt_dict[vid][8]:
        raise ValueError('Please update the latest trending date!')
    time = latest_date - yt_dict[vid][8]
    if time <= timedelta(days=6):
        trending_1w.append(vid)
    if time <= timedelta(days=30):
        trending_1m.append(vid)
    if time <= timedelta(days=90):
        trending_3m.append(vid)
    
trending_idx = {'1w': tuple(trending_1w), '1m': tuple(trending_1m), '3m': tuple(trending_3m)}

In [17]:
# trending_idx

In [18]:
data = data_re2.drop(['trending_date'],axis=1)
yout = data.set_index('video_id').T.to_dict('list')

# yt_dict = {key=video_id: value=(a tuple of attributes)}
for key in yout:
    yout[key] = tuple(yout[key])
    
import pickle
with open("./index/youtrend.pickle","wb") as file_to_write:
    pickle.dump(yout, file_to_write)

<ipython-input-18-59e62354fafc>:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  yout = data.set_index('video_id').T.to_dict('list')


In [19]:
with open('./index/categoryIdx.pickle', 'wb') as file:
    pickle.dump(category_idx, file, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
with open('./index/trendingIdx.pickle', 'wb') as file:
    pickle.dump(trending_idx, file, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
with open('./index/countryIdx.pickle', 'wb') as file:
    pickle.dump(country_idx, file, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
with open('./index/langIdx.pickle', 'wb') as file:
    pickle.dump(lang_idx, file, protocol=pickle.HIGHEST_PROTOCOL)